# Data Collection

### Riot - Data Dragon

In [1]:
import requests
import json
import os

# Data Dragon URL for latest version
VERSIONS_URL = "https://ddragon.leagueoflegends.com/api/versions.json"
BASE_URL = "https://ddragon.leagueoflegends.com/cdn"

# Get latest game version
def get_latest_version():
    res = requests.get(VERSIONS_URL)
    res.raise_for_status()
    versions = res.json()
    return versions[0]

# Load champion data
def get_champion_data(version):
    champ_list_url = f"{BASE_URL}/{version}/data/en_US/champion.json"
    res = requests.get(champ_list_url)
    res.raise_for_status()
    champions = res.json()["data"]
    return champions

# Get full details for each champion
def get_detailed_champion_data(version, champ_id):
    detail_url = f"{BASE_URL}/{version}/data/en_US/champion/{champ_id}.json"
    res = requests.get(detail_url)
    res.raise_for_status()
    return res.json()["data"][champ_id]

# Extract relevant text and save
def build_champion_text_dataset():
    version = get_latest_version()
    champions = get_champion_data(version)
    
    all_champions = []
    
    for champ_key in champions:
        champ_id = champions[champ_key]["id"]
        detail = get_detailed_champion_data(version, champ_id)
        
        name = detail["name"]
        title = detail["title"]
        blurb = detail["blurb"]
        
        passive = detail["passive"]["description"]
        spells = [spell["description"] for spell in detail["spells"]]
        
        full_text = f"{name}, {title}. {blurb} Passive: {passive} " + " ".join([f"Spell: {s}" for s in spells])
        
        all_champions.append({
            "name": name,
            "title": title,
            "blurb": blurb,
            "passive": passive,
            "spells": spells,
            "full_text": full_text
        })
    
    # Save as JSON
    with open("riot_champion_data.json", "w", encoding="utf-8") as f:
        json.dump(all_champions, f, indent=2, ensure_ascii=False)

    print(f"Saved data for {len(all_champions)} champions.")

# Run it
build_champion_text_dataset()


Saved data for 170 champions.


### LoL Wiki

In [3]:
import requests
from bs4 import BeautifulSoup
import json
import time

def scrape_lol_wiki(champion_name):
    url_name = champion_name.replace(" ", "_")
    url = f"https://leagueoflegends.fandom.com/wiki/{url_name}"
    headers = {"User-Agent": "Mozilla/5.0"}
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch {champion_name}: {e}")
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Get title from page header
    title_el = soup.find("h1", {"class": "page-header__title"})
    title = title_el.text.strip() if title_el else ""

    # Find roles
    role_box = soup.find("div", {"class": "pi-data-value pi-font"})
    roles = role_box.text.strip() if role_box else ""

    # Ability descriptions (grab first few paragraphs under abilities section)
    ability_section = soup.find("span", {"id": "Abilities"})
    if ability_section:
        ability_texts = []
        for tag in ability_section.find_parent().find_next_siblings("p", limit=6):
            ability_texts.append(tag.get_text().strip())
        abilities = " ".join(ability_texts)
    else:
        abilities = ""

    return {
        "champion": champion_name,
        "title": title,
        "roles": roles,
        "abilities": abilities
    }

def save_all_wiki_data_from_riot_json(riot_filename="riot_champion_data.json"):
    # Load all champion names from Riot JSON
    try:
        with open(riot_filename, "r", encoding="utf-8") as f:
            riot_data = json.load(f)
        champion_names = [entry["name"] for entry in riot_data]
    except Exception as e:
        print(f"Failed to load {riot_filename}: {e}")
        return

    all_data = []
    for i, name in enumerate(champion_names):
        #print(f"Scraping {name} ({i+1}/{len(champion_names)})...")         #uncomment to verify progress
        data = scrape_lol_wiki(name)
        if data:
            all_data.append(data)
            #print("Sample entry:", data)       #uncomment to verify progress
        time.sleep(1.5)  # Prevent overloading the site

    with open("wiki_champion_data.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, indent=2, ensure_ascii=False)
    print(f"\nSaved data for {len(all_data)} champions to wiki_champion_data.json.")

# Run it
save_all_wiki_data_from_riot_json()

# Printing Ahri's data as an example
with open("wiki_champion_data.json", "r", encoding="utf-8") as f:
    wiki_data = json.load(f)
    for champion in wiki_data:
        if champion["champion"] == "Ahri":
            print(json.dumps(champion, indent=2, ensure_ascii=False))
            break


Saved data for 170 champions to wiki_champion_data.json.
{
  "champion": "Ahri",
  "title": "Ahri (Character)",
  "roles": "Little Fox",
  "abilities": "Since childhood, Ahri has observed human civilization from afar until she encountered a hunter dying from a stray arrow. Curiously sensing his fading life force, Ahri got her first taste of spiritual essence and absorbed the man's memories, gaining all his knowledge and experiences. Since then, Ahri hunted down human prey for years, drunkenly drinking their essence and learning to manipulate them with her charms. She slowly felt a sense of regret as she grew empathetic from absorbing human emotions, eventually seeing how they perceived her as a monstrous fox demon. Despite this, Ahri accepted her monstrous nature and continued to feast on human essence. It wasn't until she met her first and only love, a human artist, that she began to feel true sympathy for humans and decided to completely Now, after befriending  Yasuo and the rest of

### Mobafire

In [14]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re

# Known problematic Mobafire champion URL fixes
MOBALFIRE_NAME_FIXES = {
    "Aurelion Sol": "aurelion-sol-130",
    "Bel'Veth": "belveth",
    "Cho'Gath": "chogath",
    "Dr. Mundo": "dr-mundo-26",
    "Jarvan IV": "jarvan-iv-71",
    "Kai'Sa": "kaisa",
    "Kha'Zix": "khazix",
    "Kog'Maw": "kogmaw",
    "K'Sante": "ksante",
    "Lee Sin": "lee-sin-73",
    "Master Yi": "master-yi-3",
    "Miss Fortune": "miss-fortune-59",
    "Nunu & Willump": "nunu-amp-willump-12",
    "Rek'Sai": "reksai",
    "Renata Glasc": "renata-glasc-175",
    "Tahm Kench": "tahm-kench-126",
    "Twisted Fate": "twisted-fate-28",
    "Vel'Koz": "velkoz",
    "Xin Zhao": "xin-zhao-55",
    "Wukong": "wukong-80",
}

# Format fallback names and fix special characters
def format_mobafire_name(name):
    if name in MOBALFIRE_NAME_FIXES:
        return MOBALFIRE_NAME_FIXES[name]

    fallback = name.lower().replace(" ", "-")
    fallback = fallback.replace("'", "").replace(".", "").replace("&", "and")
    fallback = re.sub(r"[^\w\-]", "", fallback)
    return fallback

# Scrape a single champion's Mobafire guide description
def scrape_mobafire_description(champion_name):
    base_name = format_mobafire_name(champion_name)
    listing_url = f"https://www.mobafire.com/league-of-legends/champion/{base_name}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        listing_response = requests.get(listing_url, headers=headers)
        listing_response.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch {champion_name} listing page: {e}")
        return {"champion": champion_name, "mobafire_description": "Error loading page"}

    listing_soup = BeautifulSoup(listing_response.text, "html.parser")

    guide_link_tag = listing_soup.find("a", class_="browse-list__guide-title")
    if not guide_link_tag or "href" not in guide_link_tag.attrs:
        return {"champion": champion_name, "mobafire_description": "No guide found"}

    guide_url = "https://www.mobafire.com" + guide_link_tag["href"]

    try:
        guide_response = requests.get(guide_url, headers=headers)
        guide_response.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch guide for {champion_name}: {e}")
        return {"champion": champion_name, "mobafire_description": "Error loading guide"}

    guide_soup = BeautifulSoup(guide_response.text, "html.parser")

    content_div = guide_soup.find("div", class_="view-guide__full")
    if not content_div:
        return {"champion": champion_name, "mobafire_description": "No content found"}

    text = content_div.get_text(separator=" ", strip=True)
    return {
        "champion": champion_name,
        "mobafire_description": text[:1000]  # Limit to first 1000 chars
    }

# Load champion names and run the full scrape
def save_all_mobafire_data_from_riot_json(riot_filename="riot_champion_data.json"):
    try:
        with open(riot_filename, "r", encoding="utf-8") as f:
            riot_data = json.load(f)
        champion_names = [entry["name"] for entry in riot_data]
    except Exception as e:
        print(f"Failed to load {riot_filename}: {e}")
        return

    all_data = []
    for i, name in enumerate(champion_names):
        #print(f"Scraping Mobafire guide for {name} ({i+1}/{len(champion_names)})...")
        data = scrape_mobafire_description(name)
        all_data.append(data)
        time.sleep(2)  # Be kind to Mobafire servers

    with open("mobafire_champion_data.json", "w", encoding="utf-8") as f:
        json.dump(all_data, f, indent=2, ensure_ascii=False)

    print(f"\nSaved data for {len(all_data)} champions to mobafire_champion_data.json.")

# Run the scraper
if __name__ == '__main__':
    save_all_mobafire_data_from_riot_json()



Saved data for 170 champions to mobafire_champion_data.json.


### Merge 3 sources

In [ ]:
import json

def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

# Load all sources
riot_data = load_json("riot_champion_data.json")
wiki_data = load_json("wiki_champion_data.json")
mobafire_data = load_json("mobafire_champion_data.json")

# Convert wiki and mobafire to dicts for faster lookup
wiki_lookup = {entry["champion"]: entry for entry in wiki_data}
mobafire_lookup = {entry["champion"]: entry for entry in mobafire_data}

# Merge data
compiled = []

for entry in riot_data:
    name = entry["name"]
    riot_text = entry.get("full_text", "")
    wiki_text = wiki_lookup.get(name, {}).get("abilities", "")
    mobafire_text = mobafire_lookup.get(name, {}).get("mobafire_description", "")

    combined_text = f"{riot_text}\n\nWIKI:\n{wiki_text}\n\nMOBAFIRE:\n{mobafire_text}"

    compiled.append({
        "champion": name,
        "combined_text": combined_text
    })

# Save the final compiled profile
with open("compiled_champion_profiles.json", "w", encoding="utf-8") as f:
    json.dump(compiled, f, indent=2, ensure_ascii=False)

print(f"Saved {len(compiled)} champions to compiled_champion_profiles.json.")


✅ Saved 170 champions to compiled_champion_profiles.json.


# Data Cleaning